In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

import numpy as np
import keras
import tensorflow
from keras.optimizers import SGD
import collections

import sys
sys.path.append("..")
from utils import *
from funcs import *

from fcnn_att import model_fcnn
from DCASE_training_functions import *

from tensorflow import ConfigProto
from tensorflow import InteractiveSession

%load_ext autoreload
%autoreload 2

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

train_csv = '/work/contractors/jtaylor/acoustic_scene_classification/data/CochlScene/train_fold_14000.tsv'
val_csv = '/work/contractors/jtaylor/acoustic_scene_classification/data/CochlScene/val_fold_3000.tsv'

feat_path = '/work/contractors/jtaylor/acoustic_scene_classification/data/CochlScene/features/logmel128_scaled/'

experiments = 'cochlscene_run_10_ordered_2/'

if not os.path.exists(experiments):
    os.makedirs(experiments)

Using TensorFlow backend.


In [2]:
#train_aug_csv = generate_train_aug_csv(train_csv, aug_csv, feat_path, aug_path, experiments)
train_aug_csv = train_csv

num_audio_channels = 1
num_freq_bin = 128
num_classes = 10
max_lr = 0.1
batch_size = 64
num_epochs = 50
mixup_alpha = 0.4
crop_length = 400
sample_num = len(open(train_aug_csv, 'r').readlines()) - 1
print(f'Numbers of training samples: {sample_num}')


# compute delta and delta delta for validation data
data_val, y_val = load_data_2020(feat_path, val_csv, num_freq_bin, 'logmel')
print(f'{len(y_val)} validation samples with values {set(y_val)}')
print(f'{collections.Counter(y_val)}\n')
print(f'validation data shape: {data_val.shape}')

data_deltas_val = deltas(data_val)
print(f'validation data deltas shape: {data_deltas_val.shape}')

data_deltas_deltas_val = deltas(data_deltas_val)
print(f'validation data delta deltas shape: {data_deltas_val.shape}')

data_val = np.concatenate((data_val[:,:,4:-4,:],data_deltas_val[:,:,2:-2,:],data_deltas_deltas_val),axis=-1)
print(f'validation data concat shape: {data_val.shape}')

y_val = keras.utils.to_categorical(y_val, num_classes)

Numbers of training samples: 14000


3000 validation samples with values {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Counter({0: 300, 3: 300, 2: 300, 1: 300, 7: 300, 4: 300, 5: 300, 8: 300, 9: 300, 6: 300})

validation data shape: (3000, 128, 431, 1)
validation data deltas shape: (3000, 128, 427, 1)
validation data delta deltas shape: (3000, 128, 427, 1)
validation data concat shape: (3000, 128, 423, 3)


In [22]:
model = model_fcnn(num_classes, input_shape=[num_freq_bin, None, 3*num_audio_channels], num_filters=[48, 96, 192], wd=0)

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer =SGD(lr=max_lr,decay=1e-6, momentum=0.9, nesterov=False),
              metrics=['accuracy'])

model.summary()

lr_scheduler = LR_WarmRestart(nbatch=np.ceil(sample_num/batch_size), Tmult=2,
                              initial_lr=max_lr, min_lr=max_lr*1e-4,
                              epochs_restart = [3.0, 7.0, 15.0, 31.0, 63.0,127.0]) 
save_path = experiments + "/model-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(save_path, monitor='val_acc', verbose=1, save_best_only=False, mode='max')
callbacks = [lr_scheduler, checkpoint]

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 128, None, 3) 0                                            
__________________________________________________________________________________________________
batch_normalization_45 (BatchNo (None, 128, None, 3) 12          input_5[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_33 (ZeroPadding2 (None, 132, None, 3) 0           batch_normalization_45[0][0]     
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 64, None, 144 10800       zero_padding2d_33[0][0]          
__________________________________________________________________________________________________
batch_norm

In [24]:
# Due to the memory limitation, in the training stage we split the training data
train_data_generator = Generator_timefreqmask_withdelta_splitted(feat_path, train_aug_csv, num_freq_bin,
                              batch_size=batch_size,
                              alpha=mixup_alpha,
                              crop_length=crop_length, splitted_num=20, classes=10)()

history = model.fit_generator(train_data_generator,
                              validation_data=(data_val, y_val),
                              epochs=num_epochs, 
                              verbose=1, 
                              workers=4,
                              max_queue_size = 100,
                              callbacks=callbacks,
                              steps_per_epoch=np.ceil(sample_num/batch_size)
                              ) 

Epoch 1/50
483/735 [==================>...........] - ETA: 4:42 - loss: 2.2798 - acc: 0.1176

KeyboardInterrupt: 